Assignment 2 - Deep Convolutional Neural Network - Cats vs Dogs

Name : Akhila Kalpuri

Date : 03-23-2024

Loading all the required libraries and functions


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import seaborn as sns
import zipfile
import io
import os
import shutil
import pathlib
%matplotlib inline
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from io import BytesIO
from zipfile import ZipFile

Importing the JSON Activation Code

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (2).json

Downloading the Data

In [ ]:
!pip install tensorflow==2.12

In [ ]:
!kaggle competitions download -c dogs-vs-cats

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


Unzipping The Data

In [ ]:
!unzip -o -qq dogs-vs-cats.zip

unzip:  cannot find or open dogs-vs-cats.zip, dogs-vs-cats.zip.zip or dogs-vs-cats.zip.ZIP.


Unzipping Train Data

In [ ]:
!unzip -o -qq train.zip

unzip:  cannot find or open train.zip, train.zip.zip or train.zip.ZIP.


In [ ]:
!pip install tensorflow==2.12

Consider the Cats & Dogs example. Start initially with a training sample of 1000, a validation sample of 500, and a test sample of 500 (like in the text). Use any technique to reduce overfitting and improve performance in developing a network that you train from scratch. What performance did you achieve?



Creating directory named cats vs dogs small to store the images into 3 subsets named train, validation and test and Dividing the training sample of 1000, a validation sample of 500, and a test sample of 500

In [ ]:
import os, shutil, pathlib
shutil.rmtree("cats_vs_dogs_small/train/cat")
original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_small")
def make_subset(subset_name, start_index, end_index):
for category in ("cat", "dog"):
dir = new_base_dir / subset_name / category
os.makedirs(dir, exist_ok = True)
fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
for fname in fnames:
shutil.copyfile(src=original_dir / fname,
dst=dir / fname)
make_subset("train", start_index=0, end_index=500)
make_subset("validation", start_index=1000, end_index=1250)
make_subset("test", start_index=1500, end_index=1750)

Data Pre-Processing and functions - Using image_dataset_from_directory to read images and functions

In [ ]:
train_dataset = image_dataset_from_directory(
new_base_dir / "train",
image_size=(180, 180),
batch_size=32)
validation_dataset = image_dataset_from_directory(
new_base_dir / "validation",
image_size=(180, 180),
batch_size=32)
test_dataset = image_dataset_from_directory(
new_base_dir / "test",
image_size=(180, 180),
batch_size=32)

Found 1000 files belonging to 2 classes.

Found 500 files belonging to 2 classes.


Found 500 files belonging to 2 classes.


Viewing the shape of the images.

In [ ]:
for data_batch, labels_batch in train_dataset:
print("data batch shape:", data_batch.shape)
print("labels batch shape:", labels_batch.shape)
break

data batch shape: (32, 180, 180, 3)

labels batch shape: (32,)



Model - 1 MaxPooling Operation with Increase in filters from 32 to 256 in 5 Input Layers : Instan- tiating a small convnet for dogs vs. cats classification


In [ ]:
#Building the model
#Instantiating a small convnet for dogs vs. cats classification
#*Model - 1 MaxPooling Operation with Increase in filters from 32 to 256 in 5␣
↪Input Layers*
inputs = keras.Input(shape=(180, 180, 3))
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

Summary of Model - 1

In [ ]:
model.summary()

In [ ]:
# Compilimg the results of the model
model.compile(loss="binary_crossentropy",
optimizer="adam",
metrics=["accuracy"])
# Saving the results of the model
callbacks = ModelCheckpoint(
filepath= "model1.keras",
save_best_only= True,
monitor= "val_loss"
)
# Fitting/Running the Model
Model_1 = model.fit(
train_dataset,
epochs= 30,
validation_data= validation_dataset,
callbacks= callbacks)

Looking at the visuals of the Training and Validation Accuracy/Loss

In [ ]:
accuracy = Model_1.history["accuracy"]
val_accuracy = Model_1.history["val_accuracy"]
loss = Model_1.history["loss"]
val_loss = Model_1.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, color="grey", label="Training Accuracy")
plt.plot(epochs, val_accuracy, color="blue",linestyle="dashed",␣
↪label="Validation Accuracy")
plt.title("Training and Validation Accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, color="grey", label="Training Loss")
plt.plot(epochs, val_loss, color="blue", linestyle="dashed", label="Validation␣
↪Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.show()

Evaluating the performance of Model_1 on test set

In [ ]:
test_model = keras.models.load_model("model1.keras")
Model1_Results = test_model.evaluate(test_dataset)
print(f'Loss: {Model1_Results[0]:.3f}')
print(f'Accuracy: {Model1_Results[1]:.3f}')

Using Measures to Avoid Overfitting Data Augmentation

In [ ]:
# To deprecate warnings that are making the output look clumsy
import logging
logging.getLogger('tensorflow').disabled = True

Using few of the techniques such as random flip, random zoom, random rotation so as to create augmented versions of the image

In [ ]:

data_augmentation = keras.Sequential(
[
layers.RandomFlip("horizontal"),
layers.RandomRotation(0.1),
layers.RandomZoom(0.2)
]
)

Looking at the augmented images

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_dataset.take(1):
for i in range(9):
augmented_images = data_augmentation(images)
ax = plt.subplot(3, 3, i + 1)
plt.imshow(augmented_images[0].numpy().astype("uint8"))
plt.axis("off")

Model - 2 MaxPooling Operation with Increase in filters from 32 to 256 in 5 Input Layers with the data being used from the Augmented Images and a droput rate of 0.5*

In [ ]:
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

Training the model 2

In [ ]:
# Compiling the model
model.compile(loss="binary_crossentropy",
optimizer="adam",
metrics=["accuracy"])
# Saving the results of the model
callbacks = ModelCheckpoint(
filepath= "model2.keras",
save_best_only= True,
monitor= "val_loss")
# Fitting/Running the Model
Model_2 = model.fit(
train_dataset,
epochs= 30,
validation_data= validation_dataset,
callbacks= callbacks)

Visualizing the Training and Validation Accuracy/Loss 2

In [ ]:
accuracy = Model_2.history["accuracy"]
val_accuracy = Model_2.history["val_accuracy"]
loss = Model_2.history["loss"]
val_loss = Model_2.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, color="grey", label="Training Accuracy")
plt.plot(epochs, val_accuracy, color="blue",linestyle="dashed",␣
↪label="Validation Accuracy")
plt.title("Training and Validation Accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, color="grey", label="Training Loss")
plt.plot(epochs, val_loss, color="blue", linestyle="dashed", label="Validation␣
↪Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.show()

Evaluating the performance of Model_2 on the test set

In [ ]:
test_model = keras.models.load_model("model2.keras")
Model2_Results = test_model.evaluate(test_dataset)
print(f'Loss: {Model2_Results[0]:.3f}')
print(f'Accuracy: {Model2_Results[1]:.3f}')

Comparing the Model 1 and Model 2 : we can clearly see that the accuracy rate of model 2 is higher than model 1



Model - 3 MaxPooling Operation with Increase in filters from 32 to 512 in 6 Input Layers with the use of Augmented Images and Dropout rate of 0.5

In [ ]:
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=512, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model summary

In [ ]:
model.summary()

Training the model 3

In [ ]:
# Compiling the Model
model.compile(loss= "binary_crossentropy",
optimizer= "adam",
metrics= ["accuracy"])
# Monitoring the best validation loss using Callbacks
callbacks = ModelCheckpoint(
filepath= "model3.keras",
save_best_only= True,
monitor= "val_loss")
# Model Fit
Model_3 = model.fit(
train_dataset,
epochs= 30,
validation_data= validation_dataset,
callbacks= callbacks)

Visualizing the Training and Validation Accuracy/Loss 2

In [ ]:
accuracy = Model_3.history["accuracy"]
val_accuracy = Model_3.history["val_accuracy"]
loss = Model_3.history["loss"]
val_loss = Model_3.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, color="grey", label="Training Accuracy")
plt.plot(epochs, val_accuracy, color="blue", linestyle="dashed",␣
↪label="Validation Accuracy")
plt.title("Training and Validation Accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, color="grey", label="Training Loss")
plt.plot(epochs, val_loss, color="blue", linestyle="dashed", label="Validation␣
↪Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.show()


Evaluating the performance of Model_2 on the test set

In [ ]:
best_model = keras.models.load_model("model3.keras")
Model3_Results = best_model.evaluate(test_dataset)
print(f'Loss: {Model3_Results[0]:.3f}')
print(f'Accuracy: {Model3_Results[1]:.3f}')

Model - 4 MaxPooling Operation with Increase in filters from 64 to 1024 in 5 Input Layers with the use of Augmented Images and Dropout rate of 0.6

In [ ]:
inputs = keras.Input(shape=(180,180,3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=512, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=1024, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.6)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

Model summary

In [ ]:
model.summary()

In [ ]:
# Compiling the Model
model.compile(loss= "binary_crossentropy",
optimizer= "adam",
metrics= ['accuracy'])
# Monitoring the best validation loss using Callbacks
callbacks = ModelCheckpoint(
filepath = "model4.keras",
save_best_only= True,
monitor= "val_loss"
)
# Model Fit
Model_4 = model.fit(
train_dataset,
epochs= 30,
validation_data= validation_dataset,
callbacks= callbacks
)

Visualizing the Training and Validation Accuracy/Loss 2

In [ ]:
accuracy = Model_4.history["accuracy"]
val_accuracy = Model_4.history["val_accuracy"]
loss = Model_4.history["loss"]
val_loss = Model_4.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, color="grey", label="Training Accuracy")
plt.plot(epochs, val_accuracy, color="blue", linestyle="dashed",␣
↪label="Validation Accuracy")
plt.title("Training and Validation Accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, color="grey", label="Training Loss")
plt.plot(epochs, val_loss, color="blue", linestyle="dashed", label="Validation␣
↪Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.show()

Evaluating the performance of Model_2 on the test set

In [ ]:
best_model = keras.models.load_model("model4.keras")
Model4_Results = best_model.evaluate(test_dataset)
print(f'Loss: {Model4_Results[0]:.3f}')
print(f'Accuracy: {Model4_Results[1]:.3f}')

Summary for Question 1: We did attempt to construct four models using a thousand as the training sample. Let's now evaluate the accuracy and loss of all four models to determine which produces the best results.



Model 1: filters from 32 to 256, 5 Input Layers


Model 2: filters from 32 to 256, 5 Input Layers, Augmented Images and Droput rate of 0.5 Model 3: filters from 32 to 512, 6 Input Layers, Augmented Images and Dropout rate of 0.5 Model 4: filters from 64 to 1024, 5 Input Layers, Augmented Images and Dropout rate of 0.6

In [ ]:
Model_1 = (0.653, 0.696)
Model_2 = (0.588, 0.714)
Model_3 = (0.620,0.674)
Model_4 = (0.604, 0.680)

In [ ]:
Models = ("Mod_1","Mod_2","Mod_3","Mod_4")
Loss = (Model_1[0],Model_2[0],Model_3[0],Model_4[0])
Accuracy = (Model_1[1],Model_2[1],Model_3[1],Model_4[1])

In [ ]:
fig, ax = plt.subplots()
ax.scatter(Loss,Accuracy)
for i, txt in enumerate(Models):
ax.annotate(txt, (Loss[i],Accuracy[i] ))
plt.title("Summary for Accuracy and Loss")
plt.ylabel("Accuracy")
plt.xlabel("Loss")
plt.show()

Conclusions: From the above graph we can conclude that model 2 is the best among all with higher accuracy and minimum loss, however model model 4 has the highest loss

Recommendation: As we can see model 2 is performing best amoung all 4 models hence we should choose model with filters from 32 to 256, 5 Input Layers, Augmented Images and Dropout rate of 0.5

Increase your training sample size. You may pick any amount. Keep the validation and test samples the same as above. Optimize your network (again training from scratch).What performance did you achieve?

Considering Training Sample – 2000

In [ ]:
import os, shutil, pathlib
original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs")
def make_subset(subset_name, start_index, end_index):
for category in ("cat", "dog"):
dir = new_base_dir / subset_name / category
os.makedirs(dir)
fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
for fname in fnames:
shutil.copyfile(src=original_dir / fname,
dst=dir / fname)
make_subset("train", start_index=0, end_index=1000)
make_subset("validation", start_index=1000, end_index=1250)
make_subset("test", start_index=1500, end_index=1750)

Data Pre-Processing: Using image_dataset_from_directory to read images

In [ ]:
train_dataset = image_dataset_from_directory(
new_base_dir / "train",
image_size=(180, 180),
batch_size=32)
validation_dataset = image_dataset_from_directory(
new_base_dir / "validation",
image_size=(180, 180),
batch_size=32)
test_dataset = image_dataset_from_directory(
new_base_dir / "test",
image_size=(180, 180),
batch_size=32)

Found 2000 files belonging to 2 classes.

Found 500 files belonging to 2 classes.

Found 500 files belonging to 2 classes.

Viewing the size of the images

In [ ]:
for data_batch, labels_batch in train_dataset:
print("data batch shape:", data_batch.shape)
print("labels batch shape:", labels_batch.shape)
break

Viewing the size of the images

In [ ]:
data_augmentation_1 = keras.Sequential(
[
layers.RandomFlip("horizontal"),
layers.RandomRotation(0.15),
layers.RandomZoom(0.25)
]
)

Model - 5 MaxPooling Operation with Increase in filters from 32 to 256 in 5 Input Layers with the data being used from the Augmented Images and a droput rate of 0.5 (Training Sample - 2000)

In [ ]:
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation_1(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)


 Model Summary


In [ ]:
model.summary()

Training the model 5

In [ ]:
# Compiling the Model
model.compile(loss= "binary_crossentropy",
optimizer= "adam",
metrics= ['accuracy'])
# Monitoring the best validation loss using Callbacks
callbacks = ModelCheckpoint(
filepath = "model5.keras",
           save_best_only= True,
           monitor= "val_loss"
)
# Model Fit
Model_5 = model.fit(
train_dataset,
epochs= 50,
validation_data= validation_dataset,
callbacks= callbacks
)

Visualizing the Training and Validation Accuracy/Loss 2

In [ ]:
accuracy = Model_5.history["accuracy"]
val_accuracy = Model_5.history["val_accuracy"]
loss = Model_5.history["loss"]
val_loss = Model_5.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, color="grey", label="Training Accuracy")
plt.plot(epochs, val_accuracy, color="blue", linestyle="dashed",␣
↪label="Validation Accuracy")
plt.title("Training and Validation Accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, color="grey", label="Training Loss")
plt.plot(epochs, val_loss, color="blue", linestyle="dashed", label="Validation␣
↪Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.show()

Evaluating the performance of Model_5 on test set

In [ ]:
best_model = keras.models.load_model("model5.keras")
Model5_Results = best_model.evaluate(test_dataset)
print(f'Loss: {Model5_Results[0]:.3f}')
print(f'Accuracy: {Model5_Results[1]:.3f}')

Summary : The accuracy of the second model, which was only generated with 1000 training examples, was 71%, but the accuracy of the same model increased to 81%, or a 10% improvement, when training samples were increased to 2000.

Model - 6 Strides Operation with Padding being “Same” with Increase in filters from 32 to 256 in 5 Input Layers with the data being used from the Augmented Images and a droput rate of 0.5 (Training Sample - 2000)

In [ ]:
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation_1(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=3, strides=2, activation="relu",␣
↪padding="same")(x)
x = layers.Conv2D(filters=64, kernel_size=3, strides=2, activation="relu",␣
↪padding="same")(x)
x = layers.Conv2D(filters=128, kernel_size=3, strides=2, activation="relu",␣
↪padding="same")(x)
x = layers.Conv2D(filters=256, kernel_size=3, strides=2, activation="relu",␣
↪padding="same")(x)
x = layers.Conv2D(filters=256, kernel_size=3, strides=2, activation="relu",␣
↪padding="same")(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

Model summary

In [ ]:
model.summary()

Training the model 6

In [ ]:
# Compiling the Model
model.compile(loss= "binary_crossentropy",
optimizer= "adam",
metrics= ['accuracy'])
# Monitoring the best validation loss using Callbacks
callbacks = ModelCheckpoint(
filepath = "model6.keras",
save_best_only= True,
monitor= "val_loss"
)
# Model Fit
Model_6 = model.fit(
train_dataset,
epochs= 50,
validation_data= validation_dataset,
callbacks= callbacks
)

Visualizing the Training and Validation Accuracy/Loss

In [ ]:
accuracy = Model_6.history["accuracy"]
val_accuracy = Model_6.history["val_accuracy"]
loss = Model_6.history["loss"]
val_loss = Model_6.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, color="grey", label="Training Accuracy")
plt.plot(epochs, val_accuracy, color="blue", linestyle="dashed",␣
↪label="Validation Accuracy")
plt.title("Training and Validation Accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, color="grey", label="Training Loss")
plt.plot(epochs, val_loss, color="blue", linestyle="dashed", label="Validation␣
↪Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.show()

In [ ]:
best_model = keras.models.load_model("model6.keras")
Model6_Results = best_model.evaluate(test_dataset)
print(f'Loss: {Model6_Results[0]:.3f}')
print(f'Accuracy: {Model6_Results[1]:.3f}')

Summary for Question 2: We did try to build 2 more models with training sample being 2000. Now lets compare the loss and Accuracy of 3 models to see which model gives better result

Model 2: filters from 32 to 256, 5 Input Layers, Augmented Images and Droput rate of 0.5, training size 1000

Model 5: filters from 32 to 256, 5 Input Layers, Augmented Images and Droput rate of 0.5, training size 2000

Model 6: filters from 32 to 256, 5 Input Layers, Augmented Images and Droput rate of 0.5, training size 2000, Padding being same

In [ ]:
Model_5 = (0.459,0.814)
Model_6 = (0.602,0.674)

In [ ]:
Models_2 = ("Mod_2","Mod_5","Mod_6")
Loss_2 = (Model_2[0],Model_5[0],Model_6[0])
Accuracy_2 = (Model_2[1],Model_5[1],Model_6[1])

In [ ]:
fig, ax = plt.subplots()
ax.scatter(Loss_2,Accuracy_2)
for i, txt in enumerate(Models_2):
ax.annotate(txt, (Loss_2[i],Accuracy_2[i] ))
plt.title("Summary for Accuracy and Loss")
plt.ylabel("Accuracy")
plt.xlabel("Loss")
plt.show()

When the models' performances were compared, it was found that the model did not gain much from using strides with padding. With the addition of a Max Pooling Layer, Model 5 outperformed the Strides model in accuracy by 14%. Additionally, an improved accuracy of 81% was attained by fine-tuning the network and expanding the training dataset from 1000 to 2000 samples.

We plotted Models 5 and 6 to answer the second question and provide a visual comparison of their performance. The graphs clearly show that Model 5 had the lowest loss of 45.9% and the highest accuracy of all the models, reaching 81%. The model performed significantly better once the training samples were increased to 2000 and various augmented photos were added.

1.	Now change your training sample so that you achieve better performance than those from Steps 1 and 2. This sample size may be larger, or smaller than those in the previous steps. The objective is to find the ideal training sample size to get best prediction results

As we saw in above graph that with the increase in training sample size the Accuracy is also increasing hence will increase the sample size to 3000 and 5000 for better performance

Training Sample 3000

In [ ]:
import os, shutil, pathlib
original_dir = pathlib.Path("train")
new_base_dir = pathlib.Path("cats_vs_dogs_1")
def make_subset(subset_name, start_index, end_index):
for category in ("cat", "dog"):
dir = new_base_dir / subset_name / category
os.makedirs(dir)
fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
for fname in fnames:
shutil.copyfile(src=original_dir / fname,
dst=dir / fname)
make_subset("train", start_index=0, end_index=1500)
make_subset("validation", start_index=1000, end_index=1250)
make_subset("test", start_index=1500, end_index=1750)

Found 3000 files belonging to 2 classes.


Found 500 files belonging to 2 classes.

In [ ]:
for data_batch, labels_batch in train_dataset: print("data batch shape:", data_batch.shape) print("labels batch shape:", labels_batch.shape) break

Using few of the techniques such as random flip, random zoom, random rotation so as to create augmented versions of the image

In [ ]:
data_augmentation_2 = keras.Sequential( [
layers.RandomFlip("horizontal"), layers.RandomRotation(0.15), layers.RandomZoom(0.25)
]
)


Model - 7 MaxPooling Operation with Increase in filters from 32 to 256 in 5 Input Layers with the data being used from the Augmented Images and a droput rate of 0.5 (Training Sample - 3000)

In [ ]:
inputs  =  keras.Input(shape=(180,  180,  3)) x = data_augmentation_2(inputs)
x = layers.Rescaling(1./255)(x)
x  =  layers.Conv2D(filters=32,  kernel_size=3,  activation="relu")(x) x  =  layers.MaxPooling2D(pool_size=2)(x)
x  =  layers.Conv2D(filters=64,  kernel_size=3,  activation="relu")(x) x  =  layers.MaxPooling2D(pool_size=2)(x)
x  =  layers.Conv2D(filters=128,  kernel_size=3,  activation="relu")(x) x  =  layers.MaxPooling2D(pool_size=2)(x)
x  =  layers.Conv2D(filters=256,  kernel_size=3,  activation="relu")(x) x  =  layers.MaxPooling2D(pool_size=2)(x)
x  =  layers.Conv2D(filters=256,  kernel_size=3,  activation="relu")(x) x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs  =  layers.Dense(1,  activation="sigmoid")(x)
model  =  keras.Model(inputs=inputs,  outputs=outputs)


In [ ]:
model.summary()

In [ ]:
# Compiling  the  Model
model.compile(loss=  "binary_crossentropy",
optimizer=  "adam", metrics=  ['accuracy'])

# Monitoring  the  best  validation  loss  using  Callbacks
callbacks = ModelCheckpoint(
filepath = "model7.keras", save_best_only= True, monitor= "val_loss"
)
# Model Fit
Model_7 = model.fit(
train_dataset, epochs= 50,
validation_data= validation_dataset, callbacks= callbacks
)


In [ ]:
accuracy = Model_7.history["accuracy"] val_accuracy = Model_7.history["val_accuracy"]

loss    =    Model_7.history["loss"] val_loss = Model_7.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs,  accuracy,  color="grey",  label="Training  Accuracy") plt.plot(epochs,  val_accuracy,  color="blue",  linestyle="dashed",␣
↪label="Validation  Accuracy")
plt.title("Training and Validation Accuracy") plt.legend()
plt.figure()
plt.plot(epochs,  loss,  color="grey",  label="Training  Loss")
plt.plot(epochs,  val_loss,  color="blue",  linestyle="dashed",  label="Validation␣
↪Loss")
plt.title("Training and Validation Loss") plt.legend()
plt.show()


In [ ]:
best_model = keras.models.load_model("model7.keras") Model7_Results = best_model.evaluate(test_dataset) print(f'Loss:  {Model7_Results[0]:.3f}') print(f'Accuracy:  {Model7_Results[1]:.3f}')

In the previous Model 6, we attempted to replace the conventional max pooling operation with strides, but the results were not as promising as expected. and in model 7 we used Maxpooling only. Therefore, we are exploring a hybrid approach that combines both max pooling and strides to evaluate the performance of this new model.

Max pooling is a downsampling operation that reduces the spatial dimensions of the feature map, aiming to capture the most prominent features while discarding less relevant information. On the other hand, strides determine the step rate of the sliding window used to extract and learn the features from the data. This hybrid approach aims to leverage the advantages of both tech- niques, potentially enhancing the model’s ability to capture intricate patterns and features while maintaining computational efficiency.

Model - 8 MaxPooling + Strides of Step-Size 2 Operation with Increase in filters from 32 to 256

in 5 Input Layers with the data being used from the Augmented Images and a droput rate of 0.5 (Training Sample - 3000)

In [ ]:
inputs  =  keras.Input(shape=(180,  180,  3)) x = data_augmentation_2(inputs)
x = layers.Rescaling(1./255)(x)
x  =  layers.Conv2D(filters=32,  kernel_size=3,  activation="relu")(x) x  =  layers.MaxPooling2D(pool_size=2,  strides=2)(x)
x  =  layers.Conv2D(filters=64,  kernel_size=3,  activation="relu")(x) x  =  layers.MaxPooling2D(pool_size=2,  strides=2)(x)
x  =  layers.Conv2D(filters=128,  kernel_size=3,  activation="relu")(x) x  =  layers.MaxPooling2D(pool_size=2,  strides=2)(x)
x  =  layers.Conv2D(filters=256,  kernel_size=3,  activation="relu")(x) x  =  layers.MaxPooling2D(pool_size=2,strides=2)(x)
x  =  layers.Conv2D(filters=256,  kernel_size=3,  activation="relu")(x) x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs  =  layers.Dense(1,  activation="sigmoid")(x)
model  =  keras.Model(inputs=inputs,  outputs=outputs)


In [ ]:
# Compiling  the  Model
model.compile(loss=  "binary_crossentropy",
optimizer=  "adam", metrics=  ['accuracy'])

# Monitoring  the  best  validation  loss  using  Callbacks
callbacks = ModelCheckpoint(
filepath = "model8.keras", save_best_only= True, monitor= "val_loss"
)
# Model Fit
Model_8 = model.fit(
train_dataset, epochs= 50,
validation_data= validation_dataset, callbacks= callbacks
)


In [ ]:
accuracy = Model_8.history["accuracy"] val_accuracy = Model_8.history["val_accuracy"]

loss    =    Model_8.history["loss"] val_loss = Model_8.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs,  accuracy,  color="grey",  label="Training  Accuracy") plt.plot(epochs,  val_accuracy,  color="blue",  linestyle="dashed",␣
↪label="Validation  Accuracy")
plt.title("Training and Validation Accuracy") plt.legend()
plt.figure()
plt.plot(epochs,  loss,  color="grey",  label="Training  Loss")
plt.plot(epochs,  val_loss,  color="blue",  linestyle="dashed",  label="Validation␣
↪Loss")
plt.title("Training and Validation Loss") plt.legend()
plt.show()


In [ ]:
best_model = keras.models.load_model("model8.keras") Model8_Results = best_model.evaluate(test_dataset) print(f'Loss:  {Model8_Results[0]:.3f}') print(f'Accuracy:  {Model8_Results[1]:.3f}')

Model - 9 MaxPooling + Strides of Step-Size 2 with Padding turned on Operation with Increase in filters from 32 to 512 in 5 Input Layers with the data being used from the Augmented Images and a droput rate of 0.5 (Training Sample - 3000)

In [ ]:
inputs  =  keras.Input(shape=(180,  180,  3)) x = data_augmentation_2(inputs)
x = layers.Rescaling(1./255)(x)
x  =  layers.Conv2D(filters=32,  kernel_size=3,  activation="relu")(x) x  =  layers.MaxPooling2D(pool_size=2,  strides=2,  padding="same")(x) x  =  layers.Conv2D(filters=64,  kernel_size=3,  activation="relu")(x) x  =  layers.MaxPooling2D(pool_size=2,  strides=2,  padding="same")(x) x  =  layers.Conv2D(filters=128,  kernel_size=3,  activation="relu")(x)
x  =  layers.MaxPooling2D(pool_size=2,  strides=2,  padding="same")(x) x  =  layers.Conv2D(filters=256,  kernel_size=3,  activation="relu")(x) x  =  layers.MaxPooling2D(pool_size=2,strides=2,  padding="same")(x)
x  =  layers.Conv2D(filters=512,  kernel_size=3,  activation="relu")(x) x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs  =  layers.Dense(1,  activation="sigmoid")(x)
model  =  keras.Model(inputs=inputs,  outputs=outputs)


In [ ]:
model.summary()

In [ ]:
# Compiling  the  Model
model.compile(loss=  "binary_crossentropy",
optimizer=  "adam", metrics=  ['accuracy'])

# Monitoring  the  best  validation  loss  using  Callbacks
callbacks = ModelCheckpoint(
filepath = "model9.keras", save_best_only= True, monitor= "val_loss"
)
# Model Fit
Model_9 = model.fit(
train_dataset, epochs= 50,
validation_data= validation_dataset, callbacks= callbacks
)


In [ ]:
accuracy = Model_9.history["accuracy"] val_accuracy = Model_9.history["val_accuracy"]

loss    =    Model_9.history["loss"] val_loss = Model_9.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs,  accuracy,  color="grey",  label="Training  Accuracy") plt.plot(epochs,  val_accuracy,  color="blue",  linestyle="dashed",␣
↪label="Validation  Accuracy")
plt.title("Training and Validation Accuracy") plt.legend()
plt.figure()
plt.plot(epochs,  loss,  color="grey",  label="Training  Loss")
plt.plot(epochs,  val_loss,  color="blue",  linestyle="dashed",  label="Validation␣
↪Loss")
plt.title("Training and Validation Loss") plt.legend()
plt.show()



In [ ]:
best_model = keras.models.load_model("model9.keras") Model9_Results = best_model.evaluate(test_dataset) print(f'Loss:  {Model9_Results[0]:.3f}') print(f'Accuracy:  {Model9_Results[1]:.3f}')

Let’s see which of the models have best performance when the training sample was set to 3000. Note: Here models 8 and 9 were trained differently with strides being used with maxpooling and strides being used with maxpooling and padding turned on.

Model 6: trides Operation with Padding being “Same” ,filters from 32 to 512, 5 Input Layers, droput rate of 0.5, Training Sample - 3000

best_model = keras.models.load_model("model9.keras") Model9_Results = best_model.evaluate(test_dataset) print(f'Loss:  {Model9_Results[0]:.3f}') print(f'Accuracy:  {Model9_Results[1]:.3f}')

Model 8: MaxPooling + Strides of Step-Size 2,filters from 32 to 512, 5 Input Layers, droput rate of 0.5, Training Sample - 3000

Model 9: MaxPooling + Strides of Step-Size 2 with Padding turned on,filters from 32 to 512, 5 Input Layers, droput rate of 0.5, Training Sample - 3000

In [ ]:
Model_7 = (0.69,0.500)
Model_8 = (0.457,0.812)
Model_9 = (0.409,0.832)


In [ ]:
Models_3  =  ("Mod_6","Mod_7","Mod_8","Mod_9")
Loss_3 = (Model_6[0],Model_7[0],Model_8[0],Model_9[0]) Accuracy_3 = (Model_6[1],Model_7[1],Model_8[1],Model_9[1])

fig, ax = plt.subplots() ax.scatter(Loss_3,Accuracy_3)
for i, txt in enumerate(Models_3): ax.annotate(txt, (Loss_3[i],Accuracy_3[i] ))
plt.title("Summary  for  Accuracy  and  Loss") plt.ylabel("Accuracy")
plt.xlabel("Loss")
plt.show()


Here we can clearly see that the model which was built with 5 layers using maxpooling along with strides and padding on was giving the highest accuracy i.e. 83.2 % with least loss amoung the other 2 models i.e. 40.9%.

Now, we are increasing the training sample to 5000 and building a model from scratch to check it’s performance on the unseen data.

Training Sample - 5000

In [ ]:
import os, shutil, pathlib
original_dir = pathlib.Path("train") new_base_dir = pathlib.Path("cats_vs_dogs_2")
def make_subset(subset_name, start_index, end_index):
for category in ("cat", "dog"):
dir = new_base_dir / subset_name / category os.makedirs(dir)
fnames = [f"{category}.{i}.jpg" for i in range(start_index, end_index)]
for fname in fnames: shutil.copyfile(src=original_dir  /  fname,
dst=dir / fname)

make_subset("train",  start_index=0,  end_index=2500) make_subset("validation",  start_index=1000,  end_index=1250) make_subset("test",  start_index=1500,  end_index=1750)


In [ ]:
train_dataset = image_dataset_from_directory(
new_base_dir / "train",
image_size=(180, 180),
batch_size=32)
validation_dataset = image_dataset_from_directory(
new_base_dir / "validation",
image_size=(180, 180),
batch_size=32)
test_dataset = image_dataset_from_directory(
new_base_dir / "test",
image_size=(180, 180),
batch_size=32)

Found 5000 files belonging to 2 classes.

Found 500 files belonging to 2 classes.

Found 500 files belonging to 2 classes.

In [ ]:
for data_batch, labels_batch in train_dataset:
print("data batch shape:", data_batch.shape)
print("labels batch shape:", labels_batch.shape)
break

data batch shape: (32, 180, 180, 3)labels batch shape: (32,)

In [ ]:
data_augmentation_2 = keras.Sequential(
[
layers.RandomFlip("horizontal"),
layers.RandomRotation(0.15),
layers.RandomZoom(0.25)
]
)

Model - 10 MaxPooling Operation with Increase in filters from 32 to 256 in 5 Input Layers with the data being used from the Augmented Images and a droput rate of 0.5 (Training Sample - 5000)

In [ ]:
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation_2(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation="relu")(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

In [ ]:
# Compiling the Model
model.compile(loss= "binary_crossentropy",
optimizer= "adam",
metrics= ['accuracy'])
# Monitoring the best validation loss using Callbacks
callbacks = ModelCheckpoint(
            filepath = "model7.keras",
            save_best_only= True,
            monitor= "val_loss"
            )
# Model Fit
Model_7 = model.fit(
train_dataset,
epochs= 50,
validation_data= validation_dataset,
callbacks= callbacks
)

In [ ]:
accuracy = Model_7.history["accuracy"]
val_accuracy = Model_7.history["val_accuracy"]
loss = Model_7.history["loss"]
val_loss = Model_7.history["val_loss"]
epochs = range(1, len(accuracy) + 1)
plt.plot(epochs, accuracy, color="grey", label="Training Accuracy")
plt.plot(epochs, val_accuracy, color="blue", linestyle="dashed",␣
↪label="Validation Accuracy")
plt.title("Training and Validation Accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, color="grey", label="Training Loss")
plt.plot(epochs, val_loss, color="blue", linestyle="dashed", label="Validation␣
↪Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.show()

In [ ]:
best_model = keras.models.load_model("model7.keras")
Model7_Results = best_model.evaluate(test_dataset)
print(f'Loss: {Model7_Results[0]:.3f}')
print(f'Accuracy: {Model7_Results[1]:.3f}')

Summary for Question 3,

We built four models, three of which were trained using a 3000 sample size. The best-performing model has an accuracy of 83.2%. Interestingly, the accuracy increased to 88.4% when we increased the training sample to 5000. Thus, we conclude that a training sample size of 5000 significantly improves the model's performance. As for the likely cause of the validation loss being less than the training loss, the split approach that was used is probably a factor.



Here, the validation and test sets are fixed at 500 apiece, but the training sample is almost five thousand times larger. Furthermore, it is crucial to recognize that regularizations like dropout or L1 and L2 regularizers are important during training and contribute to the training loss calculation. On the other hand, these regularizers are turned off during the validation or test phase, which could result in a smaller loss than the training loss.

1.	Repeat Steps 1-3, but now using a pretrained network. The sample sizes you use in Steps 2 and 3 for the pretrained network may be the same or different from those using the network where you trained from scratch. Again, use any and all optimization techniques to get best performance


Leveraging a Pre-Trained Model - VGG16

VGG - Model 1 (1000 Training Samples)

In [ ]:
conv_base = keras.applications.vgg16.VGG16( weights="imagenet",
include_top=False, input_shape=(180,  180,  3))


In [ ]:
 conv_base.summary()

In [ ]:
#extracting VGG  16  features  and  Labels
def get_features_and_labels(dataset): all_features = []
all_labels = []
for images, labels in dataset:
preprocessed_images = keras.applications.vgg16.preprocess_input(images) features = conv_base.predict(preprocessed_images) all_features.append(features)
all_labels.append(labels)
return np.concatenate(all_features), np.concatenate(all_labels)
train_features,   train_labels   =	get_features_and_labels(train_dataset) val_features, val_labels =	get_features_and_labels(validation_dataset) test_features,   test_labels   =	get_features_and_labels(test_dataset)


In [ ]:
train_features.shape

VGG - Model 1 Dense Layer with 256 Nodes and Droput Rate of 0.5 and optimizer being rmsprop with the Original Images

In [ ]:
# Defining and  Training  the  densely  connected  classifier # The last dense stacked layer and the classifier
inputs  =  keras.Input(shape=(5,  5,  512)) x = layers.Flatten()(inputs)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
# Compiling the  Model
model.compile(loss="binary_crossentropy",
optimizer="rmsprop", metrics=["accuracy"])

# Using CallBacks  to  monitor  the  best  val_loss
callbacks = ModelCheckpoint(
filepath="vgg_model1.keras", save_best_only=True, monitor="val_loss")

# Model Fit
VGG_Model_1 = model.fit(
train_features, train_labels, epochs= 30,
validation_data= (val_features, val_labels), callbacks= callbacks)


In [ ]:
acc = VGG_Model_1.history["accuracy"]
val_acc = VGG_Model_1.history["val_accuracy"]

loss   =   VGG_Model_1.history["loss"] val_loss = VGG_Model_1.history["val_loss"]
epochs = range(1, len(acc) + 1)
plt.plot(epochs,  acc,  color="grey",  label="Training  Accuracy") plt.plot(epochs,  val_acc,  color="blue",  linestyle="dashed",  label="Validation␣
↪Accuracy")
plt.title("Training and validation Accuracy") plt.legend()
plt.figure()
plt.plot(epochs,  loss,  color="grey",  label="Training  Loss")
plt.plot(epochs,  val_loss,  color="blue",  linestyle="dashed",  label="Validation␣
↪Loss")
plt.title("Training and validation Loss") plt.legend()
plt.show()


In [ ]:
best_model = keras.models.load_model("vgg_model1.keras") VGG_Model_1_Results = best_model.evaluate(test_features,test_labels) print(f'Loss:  {VGG_Model_1_Results[0]:.3f}')
print(f'Accuracy:  {VGG_Model_1_Results[1]:.3f}')


VGG - Model 2 (1000 Training Samples)

Only the densely linked networks and the classifier are permitted to modify their weights while the pre-trained model is set to maintain its current weights throughout training.

This method helps avoid overfitting because the pre-trained model doesn't change, giving the model a solid base. Furthermore, freezing the pre-trained model training might be very helpful when working with limited training data and processing resources.

We can output the list of trainable weights both before and after freezing the pre-trained model to show the effect of this setting.

In [ ]:
# Before  Freezing
conv_base.trainable = True
print("This is the number of trainable weights "
"before freezing the conv base:", len(conv_base.trainable_weights))


In [ ]:
# After  Freezing
conv_base.trainable = False
print("This is the number of trainable weights "
"after freezing the conv base:", len(conv_base.trainable_weights))


In [ ]:
conv_base.summary()

VGG - Model 2 Dense Layer with 256 Nodes and Droput Rate of 0.5 and optimizer being rmsprop with the Augmented Images

In [ ]:
# Data  Augmentation  -Adding  a  data  augmentation  stage  to  provide  augmented␣
↪training  samples  and  a  classifier  to  the  convolutional  base
data_augmentation = keras.Sequential( [
layers.RandomFlip("horizontal"), layers.RandomRotation(0.1), layers.RandomZoom(0.2),
]
)

# Adding the Classifier and Dense Network inputs  =  keras.Input(shape=(180,  180,  3)) x = data_augmentation(inputs)
x =  keras.applications.vgg16.preprocess_input(x) x = conv_base(x)
x   =    layers.Flatten()(x) x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs  =  layers.Dense(1,  activation="sigmoid")(x) model = keras.Model(inputs, outputs)
# Compiling  the  Model
model.compile(loss="binary_crossentropy",
optimizer="rmsprop", metrics=["accuracy"])

# Using  CallBacks  to  monitor  the  best  val_loss
callbacks = ModelCheckpoint(
filepath= "vgg_model2.keras", save_best_only= True, monitor= "val_loss")

# Model Fit
VGG_Model_2 = model.fit(
train_dataset, epochs= 30,
validation_data=validation_dataset, callbacks=callbacks)


In [ ]:
acc = VGG_Model_2.history["accuracy"]
val_acc = VGG_Model_2.history["val_accuracy"]

loss   =   VGG_Model_2.history["loss"] val_loss = VGG_Model_2.history["val_loss"]
epochs = range(1, len(acc) + 1)
plt.plot(epochs,  acc,  color="grey",  label="Training  Accuracy") plt.plot(epochs,  val_acc,  color="blue",  linestyle="dashed",  label="Validation␣
↪Accuracy")
plt.title("Training and validation Accuracy") plt.legend()
plt.figure()
plt.plot(epochs,  loss,  color="grey",  label="Training  Loss")
plt.plot(epochs,  val_loss,  color="blue",  linestyle="dashed",  label="Validation␣
↪Loss")
plt.title("Training and validation Loss") plt.legend()
plt.show()


In [ ]:
best_model = keras.models.load_model("vgg_model2.keras") VGG_Model_2_Results= best_model.evaluate(test_dataset) print(f'Loss:  {VGG_Model_2_Results[0]:.3f}') print(f'Accuracy:  {VGG_Model_2_Results[1]:.3f}')

Fine Tuning the VGG_Model_2

In [ ]:
conv_base.trainable = True
for layer in conv_base.layers[:-4]: layer.trainable = False


It is important to realize that pre-trained networks are trained to handle a variety of use cases and classifications, and are not only employed for solitary picture classification tasks. The network's first layers are good at gathering broad data, while its later levels usually focus on extracting features that are unique to the issue at hand. By choosing to freeze the first few layers, we may effectively avoid overfitting and allow the model to incorporate more complex information related to our particular categorization problem. The model is encouraged to concentrate on understanding the subtle nuances of the target classification problem by using this strategic approach.

In [ ]:
model.compile(loss="binary_crossentropy",
optimizer=keras.optimizers.RMSprop(learning_rate=1e-5), metrics=["accuracy"])

callbacks = ModelCheckpoint(
filepath="fine_tuning_vgg_model2.keras", save_best_only=True,
monitor="val_loss")

FineTuned_VGG_Model_2 = model.fit(
train_dataset, epochs=30,
validation_data=validation_dataset, callbacks=callbacks)


In [ ]:
acc = FineTuned_VGG_Model_2.history["accuracy"]
val_acc = FineTuned_VGG_Model_2.history["val_accuracy"]

loss = FineTuned_VGG_Model_2.history["loss"] val_loss = FineTuned_VGG_Model_2.history["val_loss"]
epochs = range(1, len(acc) + 1)
plt.plot(epochs,  acc,  color="grey",  label="Training  Accuracy") plt.plot(epochs,  val_acc,  color="blue",  linestyle="dashed",  label="Validation␣
↪Accuracy")
plt.title("Training and validation Accuracy") plt.legend()
plt.figure()
plt.plot(epochs,  loss,  color="grey",  label="Training  Loss")
plt.plot(epochs,  val_loss,  color="blue",  linestyle="dashed",  label="Validation␣
↪Loss")
plt.title("Training and validation Loss") plt.legend()
plt.show()


In [ ]:
best_model = keras.models.load_model("fine_tuning_vgg_model2.keras") FineTuned_VGG_Model_2_Results = best_model.evaluate(test_dataset) print(f"Loss:   {FineTuned_VGG_Model_2_Results[0]:.3f}") print(f"Accuracy: {FineTuned_VGG_Model_2_Results[1]:.3f}")

Using the pre-trained network VGG16, we created three models for the examination of the above two VGG16 models. Interestingly, we found that accuracy increased when the pre-trained network was frozen in its first layers and prevented from altering its weights during training. As such, we want to use the same methods with a training sample size of 5000 to construct two models.

VGG - Model 3 (5000 Training Samples)

In [ ]:
conv_base= keras.applications.vgg16.VGG16( weights="imagenet", include_top=False)


In [ ]:
conv_base.trainable = False

In [ ]:
 conv_base.summary()

In [ ]:
train_dataset = image_dataset_from_directory( new_base_dir / "train",
image_size=(180,  180), batch_size=32)
validation_dataset = image_dataset_from_directory( new_base_dir / "validation",
image_size=(180,  180), batch_size=32)
test_dataset = image_dataset_from_directory( new_base_dir / "test",
image_size=(180,  180), batch_size=32)


Found 5000 files belonging to 2 classes. Found 500 files belonging to 2 classes. Found 500 files belonging to 2 classes

In [ ]:
# Data Augmentation
data_augmentation_4 = keras.Sequential( [
layers.RandomFlip("horizontal"), layers.RandomRotation(0.15), layers.RandomZoom(0.25),
]
)

# Adding the Classifier and Dense Network inputs  =  keras.Input(shape=(180,  180,  3)) x = data_augmentation_4(inputs)
x =  keras.applications.vgg16.preprocess_input(x) x = conv_base(x)
x   =    layers.Flatten()(x) x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs  =  layers.Dense(1,  activation="sigmoid")(x) model = keras.Model(inputs, outputs)
# Compiling  the  Model
model.compile(loss="binary_crossentropy",
optimizer="adam",
              metrics=["accuracy"])
# Using CallBacks  to  monitor  the  best  val_loss
callbacks = ModelCheckpoint(
filepath= "vgg_model3.keras", save_best_only= True, monitor= "val_loss")

# Model Fit
VGG_Model_3 = model.fit(
train_dataset, epochs= 50,
validation_data=validation_dataset, callbacks=callbacks)



In [ ]:
acc = VGG_Model_3.history["accuracy"]
val_acc = VGG_Model_3.history["val_accuracy"]

loss   =   VGG_Model_3.history["loss"] val_loss = VGG_Model_3.history["val_loss"]
epochs = range(1, len(acc) + 1)
plt.plot(epochs,  acc,  color="grey",  label="Training  Accuracy") plt.plot(epochs,  val_acc,  color="blue",  linestyle="dashed",  label="Validation␣
↪Accuracy")
plt.title("Training and Validation Accuracy") plt.legend()
plt.figure()
plt.plot(epochs,  loss,  color="grey",  label="Training  Loss")
plt.plot(epochs,  val_loss,  color="blue",  linestyle="dashed",  label="Validation␣
↪Loss")
plt.title("Training and Validation Loss") plt.legend()
plt.show()


In [ ]:
best_model = keras.models.load_model("vgg_model3.keras") VGG_Model_3_Results = best_model.evaluate(test_dataset) print(f"Loss: {VGG_Model_3_Results[0]:.3f}") print(f"Accuracy: {VGG_Model_3_Results[1]:.3f}")

Fine Tunning VGG_Model_3 (Training Samples - 5000)

We have decided to freeze the first four layers in order to optimize VGG_Model3. By using this tactic, we hope to stop the model from overfitting and free it up to focus just on identifying the unique characteristics that are pertinent to our specific categorization task.As such, we have simultaneously made sure that the first four layers stay frozen and configured the pre-trained layers to remain unchanged during training. The model performs better with these improvements applied, especially when working with a training sample size of 5000.

In [ ]:
conv_base.trainable = True
for layer in conv_base.layers[:-4]: layer.trainable = False
model.compile(loss="binary_crossentropy",
optimizer=keras.optimizers.Adam(learning_rate=1e-5), metrics=["accuracy"])

callbacks = ModelCheckpoint(
filepath="fine_tuning_vgg_model3.keras", save_best_only=True,
monitor="val_loss")

FineTuned_VGG_Model_3 = model.fit(
train_dataset, epochs=50,
validation_data=validation_dataset, callbacks=callbacks)


In [ ]:
acc = FineTuned_VGG_Model_3.history["accuracy"]
val_acc = FineTuned_VGG_Model_3.history["val_accuracy"]

loss = FineTuned_VGG_Model_3.history["loss"] val_loss = FineTuned_VGG_Model_3.history["val_loss"]
epochs = range(1, len(acc) + 1)
plt.plot(epochs,  acc,  color="grey",  label="Training  Accuracy") plt.plot(epochs,  val_acc,  color="blue",  linestyle="dashed",  label="Validation␣
↪Accuracy")
plt.title("Training and validation Accuracy") plt.legend()
plt.figure()
plt.plot(epochs,  loss,  color="grey",  label="Training  Loss")
plt.plot(epochs,  val_loss,  color="blue",  linestyle="dashed",  label="Validation␣
↪Loss")
plt.title("Training and validation Loss") plt.legend()
plt.show()


In [ ]:
best_model = keras.models.load_model("fine_tuning_vgg_model3.keras") FineTuned_VGG_Model_3_Results = best_model.evaluate(test_dataset) print(f"Loss:   {FineTuned_VGG_Model_3_Results[0]:.3f}") print(f"Accuracy: {FineTuned_VGG_Model_3_Results[1]:.3f}")

After building a total of 15 models—two of which are optimized copies of the original models—we are now prepared to do a comparative study in order to identify the top models in two different categories: Pre-Trained Models and Scratch Models. Our immediate goal is to assess which scratch-built model performs the best. The evaluation comprises a comparison of the accuracy and loss metrics of the ten models that were constructed using four distinct training data. Finding the ideal training sample size for the classification of cats and dogs is the main goal.



Model 1: filters from 32 to 256, 5 Input Layers
Model 2: filters from 32 to 256, 5 Input Layers, Augmented Images and Droput rate of 0.5 Model 3: filters from 32 to 512, 6 Input Layers, Augmented Images and Dropout rate of 0.5 Model 4: filters from 64 to 1024, 5 Input Layers, Augmented Images and Dropout rate of 0.6
Model 5: filters from 32 to 256, 5 Input Layers, Augmented Images and Droput rate of 0.5, training size 2000
Model 6: filters from 32 to 256, 5 Input Layers, Augmented Images and Droput rate of 0.5, training size 2000, Padding being same
Model 7: MaxPooling Operation,filters from 32 to 512, 5 Input Layers, Augmented Images, droput rate of 0.5, Training Sample - 3000
Model 8: MaxPooling + Strides of Step-Size 2,filters from 32 to 512, 5 Input Layers, Augmented Images, droput rate of 0.5, Training Sample - 3000
Model 9: MaxPooling + Strides of Step-Size 2 with Padding turned on,filters from 32 to 512, 5 Input Layers,Augmented Images, droput rate of 0.5, Training Sample - 3000
Model 10: filters from 32 to 512, 5 Input Layers,Augmented Images, droput rate of 0.5, Training


Sample - 500


In [ ]:
# Scratch  Models
Model_1 = (0.653, 0.696)
Model_2 = (0.588, 0.714)
Model_3 = (0.620,0.674)
Model_4 = (0.604, 0.680)
Model_5 = (0.459,0.814)
Model_6 = (0.602,0.674)
Model_7 = (0.69,0.500)
Model_8 = (0.457,0.812)
Model_9 = (0.409,0.832)
Model_10 = (0.283,0.884)


In [ ]:


Models_4  =␣
↪("Mod_1","Mod_2","Mod_3","Mod_4","Mod_5","Mod_6","Mod_7","Mod_8","Mod_9","Mod_10")
Loss_4  =␣
↪(Model_1[0],Model_2[0],Model_3[0],Model_4[0],Model_5[0],Model_6[0],Model_7[0],Model_8[0],Mo
Accuracy_4  =␣
↪(Model_1[1],Model_2[1],Model_3[1],Model_4[1],Model_5[1],Model_6[1],Model_7[1],Model_8[1],Mo

fig, ax = plt.subplots() ax.scatter(Loss_4,Accuracy_4)
for i, txt in enumerate(Models_4): ax.annotate(txt, (Loss_4[i],Accuracy_4[i] ))
plt.title("Summary  for  Accuracy  and  Loss") plt.ylabel("Accuracy")
plt.xlabel("Loss")
plt.show()


Out of all the scratch models, Model_10, which was trained on 5000 samples, turned out to be the best model with an impressive 88.4% accuracy and a 28.3% loss on the test set. A five-layer architecture with filters ranging from 32 to 256 was used to build Model 10. Augmented photos were integrated into the model during training, along with a max-pooling layer and a 0.5 dropout rate.


following, we created five models with the pre-trained vgg16 network. The first three were created with a sample size of 1000 and an optimizer named rmsprop, while the following two were created with a sample size of 5000 and an optimizer named Adam.

VGG 1: filters from 32 to 256, 5 Input Layers
VGG 2: filters from 32 to 256, 5 Input Layers, Augmented Images and Droput rate of 0.5 VGG 3: filters from 32 to 512, 6 Input Layers, Augmented Images and Dropout rate of 0.5 VGG 4: VGG - Model 3 (5000 Training Samples)
VGG 5: Fine Tunning VGG_Model_3 (Training Samples - 5000)


In [ ]:
# Pre-Trained  Models
VGG_Model_1 = (0.00,100)
VGG_Model_2 = (2.6,99.8)
FineTuned_VGG_Model_2 = (0.00,100)
VGG_Model_3 = (13.4,98.4)
FineTuned_VGG_Model_3 = (0.00,100)
Models_5  =  ("VGG_1","VGG_2","VGG_3","VGG_4","VGG_5")
Loss_5  =␣
↪(VGG_Model_1[0],VGG_Model_2[0],FineTuned_VGG_Model_2[0],VGG_Model_3[0],FineTuned_VGG_Model_
Accuracy_5  =␣
↪(VGG_Model_1[1],VGG_Model_2[1],FineTuned_VGG_Model_2[1],VGG_Model_3[1],FineTuned_VGG_Model_


fig, ax = plt.subplots() ax.scatter(Loss_5,Accuracy_5)
for i, txt in enumerate(Models_5): ax.annotate(txt, (Loss_5[i],Accuracy_5[i] ))
plt.title("Summary  for  Accuracy  and  Loss") plt.ylabel("Accuracy")
plt.xlabel("Loss")
plt.show()


In [ ]:
acc = FineTuned_VGG_Model_2.history["accuracy"]
val_acc = FineTuned_VGG_Model_2.history["val_accuracy"]
loss = FineTuned_VGG_Model_2.history["loss"]
val_loss = FineTuned_VGG_Model_2.history["val_loss"]
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, color="grey", label="Training Accuracy")
plt.plot(epochs, val_acc, color="blue", linestyle="dashed", label="Validation␣
↪Accuracy")
plt.title("Training and validation Accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, color="grey", label="Training Loss")
plt.plot(epochs, val_loss, color="blue", linestyle="dashed", label="Validation␣
↪Loss")
plt.title("Training and validation Loss")
plt.legend()
plt.show()

In [ ]:
best_model = keras.models.load_model("fine_tuning_vgg_model2.keras")
FineTuned_VGG_Model_2_Results = best_model.evaluate(test_dataset)
print(f"Loss: {FineTuned_VGG_Model_2_Results[0]:.3f}")
print(f"Accuracy: {FineTuned_VGG_Model_2_Results[1]:.3f}")

Using the pre-trained network VGG16, we created three models for the examination of the above two VGG16 models. Interestingly, we found that accuracy increased when the pre-trained network was frozen in its first layers and prevented from altering its weights during training. As such, we want to use the same methods with a training sample size of 5000 to construct two models.

VGG - Model 3 (5000 Training Samples)

In [ ]:
conv_base= keras.applications.vgg16.VGG16(
weights="imagenet",
include_top=False)
conv_base.trainable = False

In [ ]:
conv_base.summary()

In [ ]:
train_dataset = image_dataset_from_directory(
new_base_dir / "train",
image_size=(180, 180),
batch_size=32)
validation_dataset = image_dataset_from_directory(
new_base_dir / "validation",
image_size=(180, 180),
batch_size=32)
test_dataset = image_dataset_from_directory(
new_base_dir / "test",
image_size=(180, 180),
batch_size=32)

Found 5000 files belonging to 2 classes. Found 500 files belonging to 2 classes. Found 500 files belonging to 2 classes.

In [ ]:
# Data Augmentation
data_augmentation_4 = keras.Sequential(
[
layers.RandomFlip("horizontal"),
layers.RandomRotation(0.15),
layers.RandomZoom(0.25),
]
)
# Adding the Classifier and Dense Network
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation_4(inputs)
x = keras.applications.vgg16.preprocess_input(x)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
# Compiling the Model
model.compile(loss="binary_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])
# Using CallBacks to monitor the best val_loss
callbacks = ModelCheckpoint(
filepath= "vgg_model3.keras",
save_best_only= True,
monitor= "val_loss")
# Model Fit
VGG_Model_3 = model.fit(
train_dataset,
epochs= 50,
validation_data=validation_dataset,
callbacks=callbacks)

In [ ]:
acc = VGG_Model_3.history["accuracy"]
val_acc = VGG_Model_3.history["val_accuracy"]
loss = VGG_Model_3.history["loss"]
val_loss = VGG_Model_3.history["val_loss"]
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, color="grey", label="Training Accuracy")
plt.plot(epochs, val_acc, color="blue", linestyle="dashed", label="Validation␣
↪Accuracy")
plt.title("Training and Validation Accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, color="grey", label="Training Loss")
plt.plot(epochs, val_loss, color="blue", linestyle="dashed", label="Validation␣
↪Loss")
plt.title("Training and Validation Loss")
plt.legend()
plt.show()

In [ ]:
best_model = keras.models.load_model("vgg_model3.keras")
VGG_Model_3_Results = best_model.evaluate(test_dataset)
print(f"Loss: {VGG_Model_3_Results[0]:.3f}")
print(f"Accuracy: {VGG_Model_3_Results[1]:.3f}")

Fine Tunning VGG_Model_3 (Training Samples - 5000)

We have decided to freeze the first four layers in order to optimize VGG_Model3. By using this tactic, we hope to stop the model from overfitting and free it up to focus just on identifying the unique characteristics that are pertinent to our specific categorization task.As such, we have simultaneously made sure that the first four layers stay frozen and configured the pre-trained layers to remain unchanged during training. The model performs better with these improvements applied, especially when working with a training sample size of 5000.

In [ ]:
conv_base.trainable = True
for layer in conv_base.layers[:-4]:
layer.trainable = False


In [ ]:
model.compile(loss="binary_crossentropy",
optimizer=keras.optimizers.Adam(learning_rate=1e-5),
metrics=["accuracy"])
callbacks = ModelCheckpoint(
filepath="fine_tuning_vgg_model3.keras",
save_best_only=True,
monitor="val_loss")
FineTuned_VGG_Model_3 = model.fit(
train_dataset,
epochs=50,
validation_data=validation_dataset,
callbacks=callbacks)

In [ ]:
acc = FineTuned_VGG_Model_3.history["accuracy"]
val_acc = FineTuned_VGG_Model_3.history["val_accuracy"]
loss = FineTuned_VGG_Model_3.history["loss"]
val_loss = FineTuned_VGG_Model_3.history["val_loss"]
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, color="grey", label="Training Accuracy")
plt.plot(epochs, val_acc, color="blue", linestyle="dashed", label="Validation␣
↪Accuracy")
plt.title("Training and validation Accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, color="grey", label="Training Loss")
plt.plot(epochs, val_loss, color="blue", linestyle="dashed", label="Validation␣
↪Loss")
plt.title("Training and validation Loss")
plt.legend()
plt.show()

In [ ]:
best_model = keras.models.load_model("fine_tuning_vgg_model3.keras")
FineTuned_VGG_Model_3_Results = best_model.evaluate(test_dataset)
print(f"Loss: {FineTuned_VGG_Model_3_Results[0]:.3f}")
print(f"Accuracy: {FineTuned_VGG_Model_3_Results[1]:.3f}")

After building a total of 15 models—two of which are optimized copies of the original models—we are now prepared to do a comparative study in order to identify the top models in two different categories: Pre-Trained Models and Scratch Models. Our immediate goal is to assess which scratch-built model performs the best. The evaluation comprises a comparison of the accuracy and loss metrics of the ten models that were constructed using four distinct training data. Finding the ideal training sample size for the classification of cats and dogs is the main goal.

Model 1: filters from 32 to 256, 5 Input Layers
Model 2: filters from 32 to 256, 5 Input Layers, Augmented Images and Droput rate of 0.5 Model 3: filters from 32 to 512, 6 Input Layers, Augmented Images and Dropout rate of 0.5 Model 4: filters from 64 to 1024, 5 Input Layers, Augmented Images and Dropout rate of 0.6
Model 5: filters from 32 to 256, 5 Input Layers, Augmented Images and Droput rate of 0.5, training size 2000
Model 6: filters from 32 to 256, 5 Input Layers, Augmented Images and Droput rate of 0.5, training size 2000, Padding being same
Model 7: MaxPooling Operation,filters from 32 to 512, 5 Input Layers, Augmented Images, droput rate of 0.5, Training Sample - 3000
Model 8: MaxPooling + Strides of Step-Size 2,filters from 32 to 512, 5 Input Layers, Augmented Images, droput rate of 0.5, Training Sample - 3000
Model 9: MaxPooling + Strides of Step-Size 2 with Padding turned on,filters from 32 to 512, 5 Input Layers,Augmented Images, droput rate of 0.5, Training Sample - 3000
Model 10: filters from 32 to 512, 5 Input Layers,Augmented Images, droput rate of 0.5, Training






Sample - 500

Out of all the scratch models, Model_10, which was trained on 5000 samples, turned out to be the best model with an impressive 88.4% accuracy and a 28.3% loss on the test set. A five-layer architecture with filters ranging from 32 to 256 was used to build Model 10. Augmented photos were integrated into the model during training, along with a max-pooling layer and a 0.5 dropout rate.

following, we created five models with the pre-trained vgg16 network. The first three were created with a sample size of 1000 and an optimizer named rmsprop, while the following two were created with a sample size of 5000 and an optimizer named Adam.

VGG 1: filters from 32 to 256, 5 Input Layers
VGG 2: filters from 32 to 256, 5 Input Layers, Augmented Images and Droput rate of 0.5 VGG 3: filters from 32 to 512, 6 Input Layers, Augmented Images and Dropout rate of 0.5 VGG 4: VGG - Model 3 (5000 Training Samples)
VGG 5: Fine Tunning VGG_Model_3 (Training Samples - 5000)


In [ ]:
# Pre-Trained Models
VGG_Model_1 = (0.00,100)
VGG_Model_2 = (2.6,99.8)
FineTuned_VGG_Model_2 = (0.00,100)
VGG_Model_3 = (13.4,98.4)
FineTuned_VGG_Model_3 = (0.00,100)

In [ ]:
Models_5 = ("VGG_1","VGG_2","VGG_3","VGG_4","VGG_5")
Loss_5 =␣
↪(VGG_Model_1[0],VGG_Model_2[0],FineTuned_VGG_Model_2[0],VGG_Model_3[0],FineTuned_VGG_Model_3[Accuracy_5 =␣
↪(VGG_Model_1[1],VGG_Model_2[1],FineTuned_VGG_Model_2[1],VGG_Model_3[1],FineTuned_VGG_Model_3[fig, ax = plt.subplots()
ax.scatter(Loss_5,Accuracy_5)
for i, txt in enumerate(Models_5):
ax.annotate(txt, (Loss_5[i],Accuracy_5[i] ))
plt.title("Summary for Accuracy and Loss")
plt.ylabel("Accuracy")
plt.xlabel("Loss")
plt.show()


When it came to pre-trained models, the two best performers were Fine- Tuned_VGG_Model_2 and FineTuned_VGG_Model_3, or Model_5and model_3.


with a meagre 0.00% loss and an astounding 100% accuracy. This model was built with 2000 and 5000 training samples, and it was tuned at a learning rate of 0.000001 using the Adam optimizer.


Conclusion : According to the aforementioned analysis, a model's accuracy is closely related to the volume of training data and the underlying architecture, especially when the model is trained directly from its own data. On the other hand, if a pretrained model is used, the accuracy depends on the particular test set that is being assessed. It's important to keep in mind that some sample sets could be more difficult to work with than others, and strong results on one set might not translate to all other sets.


Scrach Model : The profound impact on test accuracy stems from both the size of the training dataset and the chosen model architecture. Notably, integrating contemporary architectural features like residual connections, batch normalization, and depthwise separable convolutions into a basic model, alongside employing data augmentation and dropout techniques, significantly boosted test accuracy. Furthermore, expanding the training dataset from 1,000 to 3,000 samples led to substantial improvements in accuracy. Further increasing the dataset to 6,000 samples resulted in test accuracy levels similar to pretrained models. This phenomenon highlights overfitting, where a lack of samples limits the model's ability to generalize. Increasing the dataset size broadens the model's exposure to the underlying data distribution, enhancing test accuracy.

Pretrained network : However, even while test accuracy is highest when using a pretrained network, the size of the training sample has little bearing on the results. This is a result of the accuracy-measuring model not being trained using the data it handles. Furthermore, methods like data augmentation and fine-tuning do not significantly improve accuracy because the original pretrained model already has high-performance accuracy, owing to the large size of the dataset used in the pretrained VGG16 model (more than 500 MB in size and more than 138 million parameters).

Recommendations: With a rise in training data from 1,000 to 3,000 samples, the simple scratch model's accuracy increased significantly, from 0.696 to 0.88.4. A test accuracy of about 83.2% was obtained using 3,000 training samples, three contemporary architectures, data augmentation, and dropout approaches. Moreover, the test accuracy was 88% when 5,000 samples were used to train the model. Because Model 10 is the most accurate of the scratch models, it is advised to use it first.

Since we achieved 100% accuracy with the pre-tuned model, I also think that using pretrained models isn't always the best option. Context-independent reasoning states that the background or context of an image does not influence the ability to discriminate between a dog and a cat in the images used for this exercise. Therefore, as long as the sample photos belong to the same category as the target used to train the model, a pretrained model that can identify images can be applied to any image differentiation job.